In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data_raw = pd.read_parquet("datasets/result-1681818213560.parquet.gzip")\
    .drop_duplicates(subset=None, keep='first', inplace=False)
data_raw.info()

1. ebs_Q_Charge ebs_Q_Discharge BCM_ebs_I_BATT  dcdc_OverTemperature 一直为 0.0，同时又不是状态值，直接删除这些特征
2. IC_LowBatteryvoltage  DCDC_realInputVoltage  DCDC_realInputCurrent 全为空，直接删除这些特征
3. DCDC_MCU_HVIL_Warning DCDC_BMS_HVIL_Warning DCDC_HV_InterlockStatus1 DCDC_HV_InterlockStatus dcdc_deratingStatus 作为状态值先保留
4. BCM_ebs_U_BATT 中间存在 3.0 的值将这部分提取出来进一步观察

In [ ]:
data_raw.describe()

In [ ]:
# 移除一直为空的数值
drop_columns = ["ebs_Q_Charge", "ebs_Q_Discharge", "BCM_ebs_I_BATT", "dcdc_OverTemperature","IC_LowBatteryvoltage", "DCDC_realInputVoltage", "DCDC_realInputCurrent"]
data_raw = data_raw.drop(columns=drop_columns)
data_raw.describe()

In [ ]:
pd.DataFrame(data_raw["BCM_ebs_U_BATT"].value_counts()).sort_values(by='BCM_ebs_U_BATT')

In [ ]:
pd.DataFrame(data_raw["dcdc_realOutputVoltage"].value_counts()).sort_values(by='dcdc_realOutputVoltage')

In [ ]:
import matplotlib.pyplot as plt
data_raw.dcdc_realOutputVoltage.hist(bins=1000, figsize=(15,3))
plt.show()

In [ ]:
# 清除异常数据之后的数据
data_raw = data_raw.fillna(method='ffill')
data = data_raw[(data_raw.BCM_ebs_U_BATT > 3. ) & (data_raw.BCM_ebs_U_BATT < 18.)]
data.info()

In [ ]:
data[data["BCM_ebs_U_BATT"] < 10.5]["vin"].value_counts()

In [ ]:
data[data["dcdc_realOutputVoltage"] < 10.5]["vin"].value_counts()

In [ ]:
from  pandas.plotting import scatter_matrix

attributes = ["BCM_ebs_U_BATT", "dcdc_realOutputVoltage"]

scatter_matrix(data[attributes], figsize=(6,5))

In [12]:
corr_matrix = data.corr()
corr_matrix["BCM_ebs_U_BATT"].sort_value(ascending=False)

ValueError: could not convert string to float: 'LM8F7D898MA008093'